# ECOMMERCE ANALYSIS IN GOOGLE BIGQUERY

## INTRODUCTION

A newly available e-commerce Dataset that has millions of Google Analytics records for the Google Merchandise Store has been loaded into BigQuery. While imagining I am working with real-world scenarios, I will query the necessary data for a specific problem or probably needed for further analysis. The table below shows the schema from the all_sessions table.

- `fullVisitorId` (uint64): A unique identifier for each user of the website.
- `channelGrouping` (object): The channel grouping for the session.
- `time` (int64): The timestamp of the session.
- `country` (object): The country of the user.
- `city` (object): The city of the user.
- `totalTransactionRevenue` (float64): The total transaction revenue for the session.
- `transactions` (float64): The number of transactions for the session.
- `timeOnSite` (int64): The time spent on the website for the session.
- `pageviews` (int64): The number of pageviews for the session.
- `sessionQualityDim` (float64): The session quality dimension for the session.
- `date` (int64): The date of the session.
- `visitId` (int64): The visit ID for the session.
- `type` (object): The type of the session.
- `productRefundAmount` (float64): The refund amount for the product.
- `productQuantity` (float64): The quantity of the product.
- `productPrice` (int64): The price of the product.
- `productRevenue` (float64): The revenue generated by the product.
- `productSKU` (int64): The SKU of the product.
- `v2ProductName` (object): The name of the product.
- `v2ProductCategory` (object): The category of the product.
- `productVariant` (object): The variant of the product.
- `currencyCode` (object): The currency code used for the transaction.
- `itemQuantity` (float64): The quantity of the item.
- `itemRevenue` (float64): The revenue generated by the item.
- `transactionRevenue` (float64): The revenue generated by the transaction.
- `transactionId` (float64): The ID of the transaction.
- `pageTitle` (float64): The title of the page.
- `searchKeyword` (float64): The search keyword used.
- `pagePathLevel1` (object): The first level of the page path.
- `eCommerceAction_type` (int64): The type of the eCommerce action.
- `eCommerceAction_step` (int64): The step of the eCommerce action.
- `eCommerceAction_option` (float64): The option of the eCommerce action.

## EXPLORATORY DATA ANALYSIS

In [15]:
SELECT * FROM `data-to-insights.ecommerce.all_sessions`
LIMIT 10

,fullVisitorId,channelGrouping,time,country,city,totalTransactionRevenue,transactions,timeOnSite,pageviews,sessionQualityDim,date,visitId,type,productRefundAmount,productQuantity,productPrice,productRevenue,productSKU,v2ProductName,v2ProductCategory,productVariant,currencyCode,itemQuantity,itemRevenue,transactionRevenue,transactionId,pageTitle,searchKeyword,pagePathLevel1,eCommerceAction_type,eCommerceAction_step,eCommerceAction_option
0,2515546493837534633,Organic Search,966564,Taiwan,(not set),NaN,NaN,1567,82,17.0,20170721,1500643403,PAGE,NaN,NaN,0,NaN,9180781,Suitcase Organizer Cubes,(not set),(not set),USD,NaN,NaN,NaN,NaN,NaN,NaN,/storeitem.html,0,1,NaN
1,9361741997835388618,Organic Search,157377,France,not available in demo dataset,NaN,NaN,321,8,NaN,20170413,1492114470,PAGE,NaN,NaN,0,NaN,9180793,26 oz Double Wall Insulated Bottle,(not set),(not set),USD,NaN,NaN,NaN,NaN,NaN,NaN,/storeitem.html,0,1,NaN
2,7313828956068851679,Referral,228279,United States,San Francisco,NaN,NaN,927,11,63.0,20170727,1501198444,PAGE,NaN,NaN,0,NaN,9180838,Metal Texture Roller Pen,(not set),(not set),USD,NaN,NaN,NaN,NaN,NaN,NaN,/storeitem.html,0,1,NaN
3,6036794406403793540,Organic Search,1615618,United States,Boulder,NaN,NaN,1616,13,38.0,20170731,1501522359,PAGE,NaN,NaN,0,NaN,9180838,Metal Texture Roller Pen,(not set),(not set),USD,NaN,NaN,NaN,NaN,NaN,NaN,/storeitem.html,0,1,NaN
4,7847280609739507227,Organic Search,37832,Canada,not available in demo dataset,NaN,NaN,1222,45,53.0,20170728,1501300311,PAGE,NaN,NaN,0,NaN,9180838,Metal Texture Roller Pen,(not set),(not set),USD,NaN,NaN,NaN,NaN,NaN,NaN,/storeitem.html,0,1,NaN
5,1371078687799372953,Referral,478901,United States,Ann Arbor,67570000.0,1.0,1569,48,87.0,20170726,1501111386,PAGE,NaN,NaN,0,NaN,9180838,Metal Texture Roller Pen,(not set),(not set),USD,NaN,NaN,NaN,NaN,NaN,NaN,/storeitem.html,0,1,NaN
6,8781227467221587011,Referral,119567,United States,Mountain View,24710000.0,1.0,1007,35,71.0,20170727,1501203918,PAGE,NaN,NaN,0,NaN,9180838,Metal Texture Roller Pen,(not set),(not set),USD,NaN,NaN,NaN,NaN,NaN,NaN,/storeitem.html,0,1,NaN
7,7328657227470299189,Organic Search,1115991,India,not available in demo dataset,NaN,NaN,3326,41,NaN,20170305,1488700723,PAGE,NaN,NaN,0,NaN,9180842,Maze Pen,(not set),(not set),USD,NaN,NaN,NaN,NaN,NaN,NaN,/storeitem.html,0,1,NaN
8,603507101721972238,Referral,63158,United States,not available in demo dataset,NaN,NaN,372,20,NaN,20170323,1490286223,PAGE,NaN,NaN,0,NaN,9180844,Gunmetal Roller Ball Pen,(not set),(not set),USD,NaN,NaN,NaN,NaN,NaN,NaN,/storeitem.html,0,1,NaN
9,2410100047658323318,Direct,487916,New Zealand,not available in demo dataset,NaN,NaN,488,6,NaN,20170306,1488836135,PAGE,NaN,NaN,0,NaN,9180844,Gunmetal Roller Ball Pen,(not set),(not set),USD,NaN,NaN,NaN,NaN,NaN,NaN,/storeitem.html,0,1,NaN


### 1. the new all_sessions table for duplicates

In [7]:
SELECT
	fullVisitorId, 
	visitId,
	date, 
	time,
	v2ProductName, 
	productSKU,
	type,
	eCommerceAction_type,
	eCommerceAction_step,
	eCommerceAction_option,
	transactionRevenue,
	transactionId,
	COUNT(*) as row_count
FROM `data-to-insights.ecommerce.all_sessions`
GROUP BY 1,2,3 ,4, 5, 6, 7, 8, 9, 10,11,12
HAVING row_count > 1 

""


No duplicates found

### 2. Write a query that shows total unique visitors

In [8]:
SELECT
  COUNT(*) AS product_views,
  COUNT(DISTINCT fullVisitorId) AS unique_visitors
FROM `data-to-insights.ecommerce.all_sessions`;

,product_views,unique_visitors
0,21493109,389934


### 3. Write a query that shows total unique visitors(fullVisitorID) by the referring site (channelGrouping)

In [9]:
SELECT
  COUNT(DISTINCT fullVisitorId) AS unique_visitors,
  channelGrouping
FROM `data-to-insights.ecommerce.all_sessions`
GROUP BY channelGrouping
ORDER BY channelGrouping DESC;

,unique_visitors,channelGrouping
0,38101,Social
1,57308,Referral
2,11865,Paid Search
3,211993,Organic Search
4,3067,Display
5,75688,Direct
6,5966,Affiliates
7,62,(Other)


### 4. Write a query to list all the unique product names (v2ProductName) alphabetically

In [10]:
SELECT
  (v2ProductName) AS ProductName
FROM `data-to-insights.ecommerce.all_sessions`
GROUP BY ProductName
ORDER BY ProductName

,ProductName
0,1 oz Hand Sanitizer
1,14oz Ceramic Google Mug
2,15 oz Ceramic Mug
3,"15"" Android Squishable - Online"
4,16 oz. Hot and Cold Tumbler
...,...
628,YouTube Women's Short Sleeve Tri-blend Badge T...
629,YouTube Women's Short Sleeve Tri-blend Badge T...
630,YouTube Womens 3/4 Sleeve Baseball Raglan Whit...
631,YouTube Wool Heather Cap Heather/Black


### 5. Write a query to list the five products with the most views (product_views) from all visitors (including people who have viewed the same product more than once) and lists the top 5 entries.

In [11]:
SELECT
  COUNT(*) AS product_views,
  (v2ProductName) AS ProductName
FROM `data-to-insights.ecommerce.all_sessions`
WHERE type = 'PAGE'
GROUP BY v2ProductName
ORDER BY product_views DESC
LIMIT 5;

,product_views,ProductName
0,316482,Google Men's 100% Cotton Short Sleeve Hero Tee...
1,221558,22 oz YouTube Bottle Infuser
2,210700,YouTube Men's Short Sleeve Hero Tee Black
3,202205,Google Men's 100% Cotton Short Sleeve Hero Tee...
4,200789,YouTube Custom Decals


### 6. Now refine the query to no longer double-count product views for visitors who have viewed a product many times. Each distinct product view should only count once per visitor.

In [12]:
WITH unique_product_views_by_person AS (
-- find each unique product viewed by each visitor
SELECT
 fullVisitorId,
 (v2ProductName) AS ProductName
FROM `data-to-insights.ecommerce.all_sessions`
WHERE type = 'PAGE'
GROUP BY fullVisitorId, v2ProductName )
-- aggregate the top viewed products and sort them
SELECT
  COUNT(*) AS unique_view_count,
  ProductName
FROM unique_product_views_by_person
GROUP BY ProductName
ORDER BY unique_view_count DESC
LIMIT 5

,unique_view_count,ProductName
0,152358,Google Men's 100% Cotton Short Sleeve Hero Tee...
1,143770,22 oz YouTube Bottle Infuser
2,127904,YouTube Men's Short Sleeve Hero Tee Black
3,122051,YouTube Twill Cap
4,121288,YouTube Custom Decals


###  7. Expand your previous query to include the total number of distinct products ordered and the total number of total units ordered (productQuantity)

In [13]:
SELECT
  COUNT(*) AS product_views,
  COUNT(productQuantity) AS orders,
  SUM(productQuantity) AS quantity_product_ordered,
  v2ProductName
FROM `data-to-insights.ecommerce.all_sessions`
WHERE type = 'PAGE'
GROUP BY v2ProductName
ORDER BY product_views DESC
LIMIT 5;

,product_views,orders,quantity_product_ordered,v2ProductName
0,316482,3158,6352,Google Men's 100% Cotton Short Sleeve Hero Tee...
1,221558,508,4769,22 oz YouTube Bottle Infuser
2,210700,949,1114,YouTube Men's Short Sleeve Hero Tee Black
3,202205,2713,8072,Google Men's 100% Cotton Short Sleeve Hero Tee...
4,200789,1703,11336,YouTube Custom Decals


### 8. Expand the query to include the average amount of product per order (total number of units ordered/total number of orders.

In [14]:
SELECT
  COUNT(*) AS product_views,
  COUNT(productQuantity) AS orders,
  SUM(productQuantity) AS quantity_product_ordered,
  SUM(productQuantity) / COUNT(productQuantity) AS avg_per_order,
  (v2ProductName) AS ProductName
FROM `data-to-insights.ecommerce.all_sessions`
WHERE type = 'PAGE'
GROUP BY v2ProductName
ORDER BY product_views DESC
LIMIT 5;

,product_views,orders,quantity_product_ordered,avg_per_order,ProductName
0,316482,3158,6352,2.011400,Google Men's 100% Cotton Short Sleeve Hero Tee...
1,221558,508,4769,9.387795,22 oz YouTube Bottle Infuser
2,210700,949,1114,1.173867,YouTube Men's Short Sleeve Hero Tee Black
3,202205,2713,8072,2.975304,Google Men's 100% Cotton Short Sleeve Hero Tee...
4,200789,1703,11336,6.656489,YouTube Custom Decals
